In [1]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head, process_data
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [2]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [3]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
mas.show_all()
exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER, FWHM, R, S
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE, OPT
--------------------------------------------------------------------------------
ID	CATEGORY	PERIOD	AMPLITUDE	MASTER	FWHM	R	S
1	600	405	45	M5	23.0	8.0	15.9
2	600	405	53	P3	25.0	19.7	3.8
3	650	452	32	K4	12.5	11.9	17.8
4	650	452	40	K5	15.0	18.0	8.8
5	750	528	24	L3	7.0	13.8	78.5
6	750	526	30	Q4	9.5	22.0	30.7
ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/
9	20-04-20	6	729	20_04_20_Q4/


In [4]:
con.commit()

In [5]:
# 
columns = 'MASTER, DATE, S, AVG(DIP), AVG(X), AVG(Y), AVG(CON), AVG(stdX), AVG(stdY), AVG(stdCON), AVG(INP), AVG(INPPX), AVG(INPPX), AVG(IBGPX)'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE NP = 80 AND OPT = 1
GROUP BY MASTER
ORDER BY DIP DESC
;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])


MASTER	DATE	S	AVG(DIP	AVG(X)	AVG(Y)	AVG(CON	AVG(std	AVG(std	AVG(std	AVG(INP	AVG(INP	AVG(INP	AVG(IBG
L3	20-04-1	78.50	740.00	57.11	9.82	3.11	22.00	1.35	0.8999	19.64	0.9066	0.9066	0.2933
Q4	20-04-2	30.70	729.00	25.58	9.68	10.18	8.12	1.51	4.65	26.32	2.85	2.85	0.2820
K5	20-03-1	8.80	658.00	20.66	12.63	5.17	3.84	1.57	2.31	15.64	1.44	1.44	0.2827
K4	20-03-1	17.80	647.00	18.97	12.46	4.27	2.46	1.40	1.36	9.48	1.15	1.15	0.2686
P3	20-02-2	3.80	615.00	5.38	6.25	7.52	1.42	1.64	2.18	4.90	1.53	1.53	0.2082
M5	20-04-1	15.90	606.00	4.25	3.72	5.97	2.19	1.58	1.51	2.51	1.41	1.41	0.2375


In [15]:
# 
columns = 'NP, MASTER, FILE, WAVELENGTH, DIP, COUNT, RATE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE, OPT'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER='K5'
ORDER BY MASTER DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

NP	MASTER	FILE	WAVELEN	DIP	COUNT	RATE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP	OPT
80	K5	raw_01_	660	658	98	33.67	19.46	12.80	5.35	249.06	12.40	8.54	0.2291	-	1
80	K5	raw_02_	658	658	116	39.15	22.51	13.03	4.85	293.32	12.93	8.48	0.2087	-	0
80	K5	raw_03_	662	658	72	38.24	22.83	12.57	5.27	287.07	18.04	11.15	0.2858	-	1
80	K5	raw_04_	664	658	83	50.00	19.70	12.53	4.90	246.85	16.49	12.48	0.3333	-	1
80	K5	raw_05_	666	658	78	42.59	16.68	12.10	6.04	201.74	17.09	10.16	0.4306	-	0
80	K5	raw_06_	668	658	53	26.25	16.31	12.94	6.47	210.97	22.46	15.32	0.4679	-	0
80	K5	raw_07_	670	658	92	39.07	16.21	13.12	5.94	212.70	21.37	14.84	0.5083	-	0
80	K5	raw_09_	656	658	77	36.97	17.39	13.01	5.93	226.31	11.65	7.99	0.2214	-	0
80	K5	raw_10_	654	658	82	51.58	19.65	13.22	5.05	259.82	13.51	7.65	0.2727	-	0
80	K5	raw_11_	650	658	52	30.16	23.50	13.97	3.91	328.33	16.52	8.42	0.3064	-	0
60	K5	raw_13_	660	658	46	21.05	16.75	12.42	5.82	208.07	12.01	7.96	0.3303	-	1
60	K5	raw_14_	658	658	59	25.00	18.03	13.11	5.26	236.34	11.28	7.93	0

In [3]:
def spr_spectrum(folder, what = 'q'):
    main_folder='C:/SPRUP_data_Jenda/2019_03_13_Jenda_microscopy/'
    with open(main_folder + folder+'spectra_{}.txt'.format(what)) as f:
        infolist = f.read()
        info = []
    for i in infolist.split('\n')[:-1]:
        info.append([float(j) for j in i.split('\t')])
    intensity = [] 
    wlength = []
    for row in info:
        wlength.append(row[4])
        intensity.append(row[5])
    return wlength, intensity

def load_sql(sql_data, columns):
    col = columns.split(', ')

    data = dict()
    for c in col:
        data[c] = []

    for row in sql_data:
        i = 0
        for r in row:
            data[col[i]].append(r)
            i+=1
    return data

def add_spr(axesspr, name):
    wlength, intensity = spr_spectrum(name)
    wlength_pbs, intensity_pbs = spr_spectrum(name, 'pbs')
    axessen = axesspr.twinx()
    
    if len(intensity) > len(intensity_pbs): length = len(intensity_pbs)
    else: length = len(intensity)
        
    sensitivity = [(intensity_pbs[i] - intensity[i])/0.005 for i in range(length )]
    
    for tick in axesspr.get_yticklines():
        tick.set_visible(False)

    for tick in axesspr.get_yticklabels():
        tick.set_visible(False)
        
    for tick in axessen.get_yticklines():
        tick.set_visible(False)

    for tick in axessen.get_yticklabels():
        tick.set_visible(False)
        
    axessen.plot(wlength, intensity, linewidth=2, color=gray, label='spectrum', ls = ':')  
    
    axesspr.plot(wlength_pbs, sensitivity, color=red, label='sensitivity', ls = ':')  
    
def data_about_master(master):
    columns = 'MASTER, WAVELENGTH, CON, INP, IBGPX, INPPX, RATE, COUNT'
    sql_data = con.execute("""
    SELECT {}
    FROM 'masters' as MAS 
    INNER JOIN 'experiments' as EXP 
    ON MAS.ID = EXP.MASTER_ID
    INNER JOIN 'measurements' as MEAS
    ON EXP.ID = MEAS.EXPERIMENT_ID
    WHERE MASTER = '{}' AND EXP_TYPE='-' AND NP=80 AND DATE IS NOT '20-04-20' 
    ORDER BY WAVELENGTH ASC
    ;
       """.format(columns, master))
    
    sql_folder = con.execute("""
    SELECT FOLDER
    FROM 'masters' as MAS 
    INNER JOIN 'experiments' as EXP 
    ON MAS.ID = EXP.MASTER_ID
    WHERE MASTER = '{}';
       """.format(master))
    
    return load_sql(sql_data, columns), [s[0] for s in sql_folder][0]

def plot_spectrum(data, folder):
    fig, axes = plt.subplots()  
#     axes.set_title('SPECTRUM')
    axes.set_title('(a)')
    axes.set_xlabel('$\lambda$ [nm]')
    axes.set_ylabel('R')
    axessen = axes.twinx()
    axessen.set_ylabel('Sensitivity [RIU$^{-1}$]')
#     axes.grid(linestyle='--')
    
    wlength, intensity = spr_spectrum(folder)
    wlength_pbs, intensity_pbs = spr_spectrum(folder, 'pbs')
    
    if len(intensity) > len(intensity_pbs): length = len(intensity_pbs)
    else: length = len(intensity)
        
    sensitivity = [(intensity_pbs[i] - intensity[i])/0.005 for i in range(length )]
    
        
    axes.plot(wlength, intensity, linewidth=2, color=gray, label='spectrum')  
    axessen.plot(wlength_pbs, sensitivity, color=red, label='sensitivity') 
    
    lgd = fig.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
#     fig.legend(loc=3, bbox_to_anchor=(0.15, 0.2, 1, 1))
#     fig.savefig(folder_save + 'plot_spectrum_'+ data['MASTER'][0] +'.png', dpi=300, bbox_inches='tight') 
    fig.savefig(folder_save + 'plot_spectrum_'+ data['MASTER'][0] +'.png', dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')    
    
def plot_contrast(data, folder):
    fig, axes = plt.subplots()  
#     axes.set_title('CONTRAST')
    axes.set_title('(b)')
    axes.set_xlabel('$\lambda$ [nm]')
    axes.set_ylabel('bg intensity [a.u.]')

    axes_ii=axes.twinx()
#     axes.grid(linestyle='--', axis = 'x')
    axes_ii.set_ylabel('NP contrast [a.u.]')
#     axes_ii.grid(linestyle='--', axis = 'y')
    
    axes_ii.plot(data['WAVELENGTH'], data['CON'], linewidth=2, color=red, label='contrast') 
    axes_ii.set_ylim(0, max( data['CON'])*1.1)
    axes.plot(data['WAVELENGTH'], data['IBGPX'], linewidth=2, color=black, label='background int.')

    add_spr(axes.twinx(), folder)
    lgd = fig.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))
    fig.savefig(folder_save + 'plot_contrast_'+ data['MASTER'][0] +'.png', dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')    
    
def plot_npi(data, folder):
    fig, axes = plt.subplots()
#     axes.set_title('NP intensity')
    axes.set_title('(c)')
    axes.set_xlabel('$\lambda$ [nm]')
    axes.set_ylabel('NP intensity per')
    axes_ii=axes.twinx()
#     axes.grid(linestyle='--', axis = 'x')
    axes_ii.set_ylabel('NP intensity per px[a.u.]')
#     axes_ii.grid(linestyle='--', axis = 'y')

    axes_ii.plot(data['WAVELENGTH'], data['INP'], linewidth=2, color=yellow, label='NP intensity per px')
    axes_ii.set_ylim(0, max( data['INP'])*1.1)
    axes.plot(data['WAVELENGTH'], data['INPPX'], linewidth=2, color=blue, label='intensity of NP')
    axes.set_ylim(0, max( data['INPPX'])*1.1)
    
    add_spr(axes.twinx(), folder)
    lgd = fig.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))
    fig.savefig(folder_save + 'plot_intensity_'+ data['MASTER'][0] +'.png', dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')    
    
def plot_rate(data, folder):
    fig, axes = plt.subplots()
    axes.set_title('(d)')
#     axes.set_title('NP binding rate & count')
    axes.set_xlabel('$\lambda$ [nm]')
    axes.set_ylabel('total count')
    axes_ii=axes.twinx()

    
#     axes.grid(linestyle='--', axis = 'x')    
    axes_ii.set_ylabel('binding rate [#/100f]')
#     axes_ii.grid(linestyle='--', axis = 'y')

    axes_ii.plot(data['WAVELENGTH'], data['RATE'], linewidth=2, color=green, label='rate')    
    axes.plot(data['WAVELENGTH'], data['COUNT'], linewidth=2, color=purple, label='count')
    axes_ii.set_ylim(0, max( data['RATE'])*1.1)
    axes.set_ylim(0, max( data['COUNT'])*1.1)
    
    add_spr(axes.twinx(), folder)
    lgd = fig.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))
    
    fig.savefig(folder_save + 'plot_binding_rate_'+ data['MASTER'][0] +'.png', dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')    
    
def plot_for_thesis(data, folder):
    fig, axes = plt.subplots()
    axes.set_title(data['MASTER'][0])
    axes.set_xlabel('$\lambda$ [nm]')
    axes.set_ylabel('contrast')
    axes_ii=axes.twinx()

    
#     axes.grid(linestyle='--', axis = 'x')    
    axes_ii.set_ylabel('binding rate [#/100f]')
                 
#     axes_ii.grid(linestyle='--', axis = 'y')

    axes_ii.plot(data['WAVELENGTH'], data['RATE'], linewidth=2, color=green, label='binding rate')    
    axes.plot(data['WAVELENGTH'], data['CON'], linewidth=2, color=red, label='contrast')
    
    axes_ii.set_ylim(0, max( data['RATE'])*1.1)
    axes.set_ylim(0, max( data['CON'])*1.1)
    
    add_spr(axes.twinx(), folder)
#     lgd = fig.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))

#     fig.savefig(folder_save + 'plot_characterization_'+ data['MASTER'][0] +'.png', dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(folder_save + 'plot_characterization_new_'+ data['MASTER'][0] +'.png', dpi=300, bbox_inches='tight')    

In [54]:
# 
columns = 'NP, FILE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'P3' AND WAVELENGTH = 612 AND NP = 80

ORDER BY NP DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

NP	FILE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP
80	raw_10_	5.06	5.98	7.86	30.26	4.53	2.27	0.1912	-
80	raw_29_	5.11	6.36	8.16	32.51	4.91	2.18	0.2032	sh
80	raw_32_	5.11	6.11	7.43	31.24	4.54	2.02	0.2021	strep
---
3


In [30]:
# 
columns = 'MEAS.ID, MASTER, NP, WAVELENGTH, FOLDER, FILE, RATE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE, OPT'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'P3' AND NP=80 AND OPT=1
ORDER BY RATE DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

MEAS.ID	MASTER	NP	WAVELEN	FOLDER	FILE	RATE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP	OPT
50	P3	80	612	20_02_2	raw_32_	168.45	5.11	6.11	7.43	31.24	4.54	2.02	0.2021	strep	1
36	P3	80	612	20_02_2	raw_10_	46.43	5.06	5.98	7.86	30.26	4.53	2.27	0.1912	-	1
44	P3	80	610	20_02_2	raw_11_	42.78	6.25	6.53	6.63	40.83	5.63	2.33	0.2363	-	1
49	P3	80	612	20_02_2	raw_29_	30.00	5.11	6.36	8.16	32.51	4.91	2.18	0.2032	sh	1
---
4


In [66]:
exp.show_all()

ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/
9	20-04-20	6	729	20_04_20_Q4/


In [96]:
con.commit()

In [12]:
folder = '20_04_20_Q4/'
file = 'raw_17_1'

columns = 'COUNT(MEAS.ID), MEAS.ID'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID 
WHERE FOLDER = '{}' AND FILE = '{}'
;
   """.format(columns, folder, file)
                  )

In [26]:
[d for d in data][0]

(3, 79)

In [20]:
b

In [28]:
columns = 'NP, WAVELENGTH, AR_TH, AR_NOISE, EXC_THS, AR_MIN, AR_COND, exp_type'
data = con.execute("""
SELECT {} FROM 'measurements'
WHERE WAVELENGTH=732
ORDER BY WAVELENGTH ASC;
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	AR_TH	AR_NOISE	EXC_THS	AR_MIN	AR_COND	exp_type
80	732	60.0	0.001	[2]	10	2	-
60	732	59.0	0.001	[2]	10	2	-
60	732	46.0	0.001	[2]	10	2	sh
40	732	12.0	0.001	[1.1]	2	2	-
80	732	100.0	0.001	[2]	2	2	-
60	732	20.0	0.001	[2]	2	2	-
80	732	40.0	0.001	[2]	2	2	strep
147	732	8.0	0.001	[2]	2	2	ps
40	732	9.0	0.001	[2]	6	2	-
80	732	31.0	0.001	[2]	6	2	sh


In [59]:
con.commit()

In [51]:
con.close()